# Time series

In [ ]:
from __future__ import division
from pandas import Series, DataFrame
import pandas as pd
from numpy.random import randn
import numpy as np
pd.options.display.max_rows = 12
np.set_printoptions(precision=4, suppress=True)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(12, 4))

## Periods and Period Arithmetic

### Creating a PeriodIndex from arrays

## Resampling and Frequency Conversion

In [ ]:
rng = pd.date_range('1/1/2000', periods=100, freq='D')
ts = Series(randn(len(rng)), index=rng)
ts.resample('M', how='mean')

In [ ]:
ts.resample('M', how='mean', kind='period')

### Downsampling

In [ ]:
rng = pd.date_range('1/1/2000', periods=12, freq='T')
ts = Series(np.arange(12), index=rng)
ts

In [ ]:
ts.resample('5min', how='sum')
# note: output changed (as the default changed from closed='right', label='right' to closed='left', label='left'

In [ ]:
ts.resample('5min', how='sum', closed='left')

In [ ]:
ts.resample('5min', how='sum', closed='left', label='left')

In [ ]:
ts.resample('5min', how='sum', loffset='-1s')

#### Open-High-Low-Close (OHLC) resampling

In [ ]:
ts.resample('5min', how='ohlc')
# note: output changed because of changed defaults

#### Resampling with GroupBy

In [ ]:
rng = pd.date_range('1/1/2000', periods=100, freq='D')
ts = Series(np.arange(100), index=rng)
ts.groupby(lambda x: x.month).mean()

In [ ]:
ts.groupby(lambda x: x.weekday).mean()

### Upsampling and interpolation

In [ ]:
frame = DataFrame(np.random.randn(2, 4),
                  index=pd.date_range('1/1/2000', periods=2, freq='W-WED'),
                  columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame

In [ ]:
df_daily = frame.resample('D')
df_daily

In [ ]:
frame.resample('D', fill_method='ffill')

In [ ]:
frame.resample('D', fill_method='ffill', limit=2)

In [ ]:
frame.resample('W-THU', fill_method='ffill')

### Resampling with periods

In [ ]:
frame = DataFrame(np.random.randn(24, 4),
                  index=pd.period_range('1-2000', '12-2001', freq='M'),
                  columns=['Colorado', 'Texas', 'New York', 'Ohio'])
frame[:5]

In [ ]:
annual_frame = frame.resample('A-DEC', how='mean')
annual_frame

In [ ]:
# Q-DEC: Quarterly, year ending in December
annual_frame.resample('Q-DEC', fill_method='ffill')
# note: output changed, default value changed from convention='end' to convention='start' + 'start' changed to span-like
# also the following cells

In [ ]:
annual_frame.resample('Q-DEC', fill_method='ffill', convention='start')

In [ ]:
annual_frame.resample('Q-MAR', fill_method='ffill')

## Time series plotting

In [ ]:
close_px_all = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
close_px = close_px_all[['AAPL', 'MSFT', 'XOM']]
close_px = close_px.resample('B', fill_method='ffill')
close_px.info()

In [ ]:
close_px['AAPL'].plot()

In [ ]:
close_px.ix['2009'].plot()

In [ ]:
close_px['AAPL'].ix['01-2011':'03-2011'].plot()

In [ ]:
appl_q = close_px['AAPL'].resample('Q-DEC', fill_method='ffill')
appl_q.ix['2009':].plot()

## Moving window functions

In [ ]:
close_px = close_px.asfreq('B').fillna(method='ffill')

In [ ]:
close_px.AAPL.plot()
pd.rolling_mean(close_px.AAPL, 250).plot()

In [ ]:
plt.figure()

In [ ]:
appl_std250 = pd.rolling_std(close_px.AAPL, 250, min_periods=10)
appl_std250[5:12]

In [ ]:
appl_std250.plot()

In [ ]:
# Define expanding mean in terms of rolling_mean
expanding_mean = lambda x: rolling_mean(x, len(x), min_periods=1)

In [ ]:
pd.rolling_mean(close_px, 60).plot(logy=True)

In [ ]:
plt.close('all')

### Exponentially-weighted functions

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, sharey=True,
                         figsize=(12, 7))

aapl_px = close_px.AAPL['2005':'2009']

ma60 = pd.rolling_mean(aapl_px, 60, min_periods=50)
ewma60 = pd.ewma(aapl_px, span=60)

aapl_px.plot(style='k-', ax=axes[0])
ma60.plot(style='k--', ax=axes[0])
aapl_px.plot(style='k-', ax=axes[1])
ewma60.plot(style='k--', ax=axes[1])
axes[0].set_title('Simple MA')
axes[1].set_title('Exponentially-weighted MA')

### Binary moving window functions

In [ ]:
close_px
spx_px = close_px_all['SPX']

In [ ]:
spx_rets = spx_px / spx_px.shift(1) - 1
returns = close_px.pct_change()
corr = pd.rolling_corr(returns.AAPL, spx_rets, 125, min_periods=100)
corr.plot()

In [ ]:
corr = pd.rolling_corr(returns, spx_rets, 125, min_periods=100)
corr.plot()

### User-defined moving window functions

In [ ]:
from scipy.stats import percentileofscore
score_at_2percent = lambda x: percentileofscore(x, 0.02)
result = pd.rolling_apply(returns.AAPL, 250, score_at_2percent)
result.plot()

## Performance and Memory Usage Notes

In [ ]:
rng = pd.date_range('1/1/2000', periods=10000000, freq='10ms')
ts = Series(np.random.randn(len(rng)), index=rng)
ts

In [ ]:
ts.resample('15min', how='ohlc').info()

In [ ]:
%timeit ts.resample('15min', how='ohlc')

In [ ]:
rng = pd.date_range('1/1/2000', periods=10000000, freq='1s')
ts = Series(np.random.randn(len(rng)), index=rng)
%timeit ts.resample('15s', how='ohlc')